In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import re

# Preparacion de la exploracion

In [4]:
test_path = '/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/raw_lichess_db/lichess_db_standard_rated_2020-02.pgn.bz2'

__CODIGO PARA GENERAR SAMPLE__

In [4]:
!bzcat {test_path} -s | head -n 2000 > ./samples/games_sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/raw_lichess_db/lichess_db_standard_rated_2020-02.pgn.bz2, output file = (stdout)


In [18]:
sample_path = '/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/samples/games_sample.csv'
sample = pd.read_csv(sample_path, sep=';', header=None)

# TRABAJO EN CURSO

In [100]:
def old_read_games(sample_df, games, eval='yes', process_games='process'):
    cols = [
        'Event', 
        'Site', 
        'Date', 
        'Round', 
        'WhiteUser', 
        'BlackUser', 
        'Result', 
        'UTCDate', 
        'UTCTime', 
        'WHiteElo', 
        'BlackElo', 
        'WhiteRatingDiff', 
        'BlackRatingDiff', 
        'ECO', 
        'Opening', 
        'TimeControl', 
        'Termination', 
        'Game'
        ]
    df = pd.DataFrame(columns=cols)
    game_row = 0
    for i in range(0, games):
        vert_game = sample_df.iloc[game_row:game_row+18]
        print(pd.Series(vert_game[0]))
        game = vert_game.transpose()

        if vert_game[0].str.find('WhiteTitle')>=0:
            pass
        elif vert_game[0].str.find('BlackTitle')>=0:
            pass
        else:
            game.columns = cols
            df = pd.concat([df, game])
        game_row += 18
    df.reset_index(drop=True, inplace=True)
    

    # Game processing
    if process_games == 'process':
            # Clean all columns but GAME, which needs to remain raw
            df.loc[:, df.columns != 'Game'] = df.loc[:, df.columns != 'Game'].applymap(lambda x: x.split(' ', 1)[1].replace('"', '')[:-1])
            
            # Format date column
            df['Date'] = pd.to_datetime(df['UTCDate'])

            # Find game type by time control and set a variable for it
            game_type = pd.DataFrame(columns=['Bullet', 'Blitz', 'Rapid', 'Classical'])
            for i in df['Event']:
                bullet = ((i.find('Bullet')>0)*1)
                blitz = ((i.find('Blitz')>0)*1)
                rapid = ((i.find('Rapid')>0)*1)
                classical = ((i.find('Classical')>0)*1)
                gamedf = pd.DataFrame([{'Bullet':bullet,
                                        'Blitz':blitz,
                                        'Rapid':rapid,
                                        'Classical':classical}])
                game_type = pd.concat([game_type, gamedf], ignore_index=True)
            df = df.join(game_type)
            
            # Final Step: Drop unuseful columns
            df.drop(['Event', 'Site', 'ECO', 'BlackRatingDiff', 'WhiteUser', 'BlackUser', 'UTCDate', 'Round'], axis=1, inplace=True)
    # Gets only evaluated games

    if eval == 'yes':
        df = df[df['Game'].apply(lambda x: x.find('eval')>0)==True]


    return df

In [149]:
def clean_line(inputstring):
    return re.findall("\"([^\"]*)\"", inputstring)[0]

In [185]:
def read_games(raw_list, eval='yes'):
    df=pd.DataFrame()
    idx = -1
    
    for line in raw_list:
        if line.startswith('[Event'):
            idx+=1
            df.loc[idx, "Event"] = clean_line(line)
        elif line.startswith('[UTCDate'):
            df.loc[idx, "Date"] = clean_line(line)
        elif line.startswith('[Result'):
            df.loc[idx, "Result"] = clean_line(line)
        elif line.startswith('[WhiteElo'):
            df.loc[idx, "WhiteELO"] = clean_line(line)
        elif line.startswith('[BlackElo'):
            df.loc[idx, "BlackELO"] = clean_line(line)
        elif line.startswith('[Opening'):
            df.loc[idx, "Opening"] = clean_line(line)
        elif line.startswith('[TimeControl'):
            df.loc[idx, "TimeControl"] = clean_line(line)
        elif line.startswith('[Termination'):
            df.loc[idx, "Termination"] = clean_line(line)
        elif line.startswith('[WhiteTitle'):
            df.loc[idx, "WhiteTitle"] = clean_line(line)
        elif line.startswith('[BlackTitle'):
            df.loc[idx, "BlackTitle"] = clean_line(line)
        elif line.startswith('1. '):
            df.loc[idx, "Game"] = str(line)
        else:
            pass
    df.fillna("-", inplace=True)
    if eval == 'yes':
        #df = df[df['Game'].apply(lambda x: x.find('eval')>0)==True]
        try:
            df = df[df['Game'].apply(lambda x: x.find('eval')>0)==True]
        except:
            df = df[df['Game'].apply(lambda x: print(x))]

    
    return df
    

In [161]:
test_sample = !bzcat {test_path} -s | head -n 2000

In [159]:
test = read_games(test_sample)

In [ ]:
test.to_csv('/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/samples/games_processed_sample.csv')

In [198]:
def bash_read_and_save(test_path, startpoint, chunksize, out_path):
    chunk = !bzcat {test_path} | tail -n +{startpoint+1} | head -n {chunksize}
    chunkdf = read_games(chunk)
    chunkdf.to_csv(out_path)

In [202]:
def process_files(dir_path, startpoint=0, chunksize=20000):
    # listamos archivos en la carpeta
    # list to store files
    file_list = []

    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            file_list.append(path)

    # iteramos para cada archivo
    for file in file_list:
        save_path = dir_path+"filtered/"+file
        try:
            !mkdir {save_path}
        except:
            pass
        test_path = dir_path+file
               
        i=True
        while i==True:
            out_path = save_path+"/"+str(startpoint)+"-"+str(startpoint+chunksize)+"_processed.csv"
            try:
                print(startpoint)
                bash_read_and_save(test_path, startpoint, chunksize, out_path)
                startpoint+=chunksize
            except Exception as e:
                print(e)
                i=False
                print("END")


In [203]:
dir_path = '/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/raw_lichess_db/'

process_files(dir_path, chunksize=1_000_000)

0
1000000


In [164]:
file_list = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        file_list.append(path)
file = file_list[0]

In [165]:
test_path = dir_path+file
save_path = dir_path+"filtered/"+file
out_path = save_path+"/"+str(0)+"-"+str(100000)+".csv"

In [166]:
out_path

'/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/raw_lichess_db/filtered/lichess_db_standard_rated_2020-02.pgn.bz2/0-100000.csv'

In [168]:
chunk = !bzcat {test_path} | tail -n +{0+1} | head -n {100_000}


In [188]:
chunkdf = read_games(chunk, eval='yes')

In [189]:
chunkdf

,Event,Result,Date,WhiteELO,BlackELO,Opening,TimeControl,Termination,Game,WhiteTitle,BlackTitle
21,Rated Blitz game,1-0,2020.02.01,1791,1829,Queen's Gambit Refused: Marshall Defense,180+2,Normal,1. d4 { [%eval 0.25] [%clk 0:03:00] } 1... d5 ...,-,-
47,Rated Blitz game,1-0,2020.02.01,2120,2104,"Sicilian Defense: Smith-Morra Gambit Accepted,...",300+0,Normal,1. e4 { [%eval 0.24] [%clk 0:05:00] } 1... c5 ...,-,-
65,Rated Rapid game,1-0,2020.02.01,1457,1521,Sicilian Defense: Old Sicilian,600+0,Normal,1. e4 { [%eval 0.24] [%clk 0:10:00] } 1... c5 ...,-,-
66,Rated Rapid game,1-0,2020.02.01,2296,2303,Queen's Pawn Game: London System,600+0,Normal,1. d4 { [%eval 0.25] [%clk 0:10:00] } 1... Nf6...,-,-
68,Rated Rapid game,0-1,2020.02.01,1547,1560,Queen's Pawn Game: Mason Attack,600+0,Normal,1. d4 { [%eval 0.25] [%clk 0:10:00] } 1... d5 ...,-,-
...,...,...,...,...,...,...,...,...,...,...,...
4922,Rated Bullet game,0-1,2020.02.01,1649,1697,St. George Defense,60+0,Normal,1. e4 { [%eval 0.24] [%clk 0:01:00] } 1... a6 ...,-,-
4933,Rated Blitz tournament https://lichess.org/tou...,1-0,2020.02.01,1810,1896,English Opening: The Whale,180+0,Normal,1. e4 { [%eval 0.24] [%clk 0:01:30] } 1... e5 ...,-,-
4946,Rated Bullet game,1-0,2020.02.01,2144,2293,Bird Opening: From's Gambit,120+1,Normal,1. f4 { [%eval -0.1] [%clk 0:02:00] } 1... e5?...,-,-
4949,Rated Rapid game,0-1,2020.02.01,1508,1730,Caro-Kann Defense,1200+0,Normal,1. e4 { [%eval 0.24] [%clk 0:20:00] } 1... c6 ...,-,-


In [187]:
chunkdf[chunkdf["Game"].isna()]

,Event,Result,Date,WhiteELO,BlackELO,Opening,TimeControl,Termination,Game,WhiteTitle,BlackTitle


In [49]:
chunkdf[(chunkdf['Round']!='[Round "-"]')&(chunkdf['Round']!='[Date "2020.02.01"]')]

,Event,Site,Date,Round,WhiteUser,BlackUser,Result,UTCDate,UTCTime,WHiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,Game
232,"[Termination ""Normal""]",1. e4 { [%clk 0:01:00] } g6 { [%clk 0:01:00] }...,"[Event ""Rated Bullet game""]","[Site ""https://lichess.org/zTySWoZ6""]","[Date ""2020.02.01""]","[Round ""-""]","[White ""changoski""]","[Black ""adtsakal""]","[Result ""1-0""]","[UTCDate ""2020.02.01""]","[UTCTime ""00:00:18""]","[WhiteElo ""1895""]","[BlackElo ""1894""]","[WhiteRatingDiff ""+6""]","[BlackRatingDiff ""-6""]","[ECO ""B30""]","[Opening ""Sicilian Defense: Old Sicilian""]","[TimeControl ""60+0""]"
233,"[Termination ""Time forfeit""]",1. e4 { [%clk 0:01:00] } c5 { [%clk 0:01:00] }...,"[Event ""Rated Bullet game""]","[Site ""https://lichess.org/JptsopxL""]","[Date ""2020.02.01""]","[Round ""-""]","[White ""nojod4s""]","[Black ""stun33""]","[Result ""0-1""]","[UTCDate ""2020.02.01""]","[UTCTime ""00:00:18""]","[WhiteElo ""2003""]","[BlackElo ""2047""]","[WhiteRatingDiff ""-5""]","[BlackRatingDiff ""+5""]","[ECO ""B07""]","[Opening ""King's Pawn Game: Maróczy Defense""]","[TimeControl ""60+0""]"
234,"[Termination ""Normal""]",1. d4 { [%clk 0:01:00] } d6 { [%clk 0:01:00] }...,"[Event ""Rated Bullet game""]","[Site ""https://lichess.org/z8VpZKkC""]","[Date ""2020.02.01""]","[Round ""-""]","[White ""joo_mourad20""]","[Black ""Jempitoveloz""]","[Result ""1-0""]","[UTCDate ""2020.02.01""]","[UTCTime ""00:00:18""]","[WhiteElo ""2079""]","[BlackElo ""2178""]","[WhiteRatingDiff ""+7""]","[BlackRatingDiff ""-8""]","[ECO ""B07""]","[Opening ""Pirc Defense""]","[TimeControl ""60+0""]"
235,"[Termination ""Normal""]",1. e4 { [%clk 0:01:00] } d6 { [%clk 0:01:00] }...,"[Event ""Rated Bullet game""]","[Site ""https://lichess.org/Dg2Jjagr""]","[Date ""2020.02.01""]","[Round ""-""]","[White ""chilestrong""]","[Black ""schubertboy""]","[Result ""1-0""]","[UTCDate ""2020.02.01""]","[UTCTime ""00:00:18""]","[WhiteElo ""1822""]","[BlackElo ""1822""]","[WhiteRatingDiff ""+6""]","[BlackRatingDiff ""-5""]","[ECO ""B01""]","[Opening ""Scandinavian Defense""]","[TimeControl ""60+0""]"
236,"[Termination ""Normal""]",1. e4 { [%eval 0.24] [%clk 0:01:00] } 1... d5 ...,"[Event ""Rated Bullet game""]","[Site ""https://lichess.org/gP5aKmVq""]","[Date ""2020.02.01""]","[Round ""-""]","[White ""nessuno88""]","[Black ""ekstravagantni""]","[Result ""1-0""]","[UTCDate ""2020.02.01""]","[UTCTime ""00:00:18""]","[WhiteElo ""1714""]","[BlackElo ""1751""]","[WhiteRatingDiff ""+6""]","[BlackRatingDiff ""-6""]","[ECO ""B23""]","[Opening ""Sicilian Defense: Grand Prix Attack""]","[TimeControl ""120+1""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,"[BlackRatingDiff ""+3""]","[ECO ""A00""]","[Opening ""Hungarian Opening: Catalan Formation""]","[TimeControl ""300+0""]","[Termination ""Time forfeit""]",1. g3 { [%clk 0:05:00] } e6 { [%clk 0:05:00] }...,"[Event ""Rated Blitz tournament https://lichess...","[Site ""https://lichess.org/E9z4VtuP""]","[Date ""2020.02.01""]","[Round ""-""]","[White ""Kavkaz-Karachay""]","[Black ""kemangbah""]","[Result ""1-0""]","[UTCDate ""2020.02.01""]","[UTCTime ""00:05:44""]","[WhiteElo ""2091""]","[BlackElo ""1558""]","[WhiteRatingDiff ""+1""]"
4996,"[BlackRatingDiff ""-1""]","[ECO ""B01""]","[Opening ""Scandinavian Defense""]","[TimeControl ""300+0""]","[Termination ""Time forfeit""]",1. e4 { [%clk 0:02:30] } d5 { [%clk 0:05:00] }...,"[Event ""Rated Bullet game""]","[Site ""https://lichess.org/C4Ju5VWy""]","[Date ""2020.02.01""]","[Round ""-""]","[White ""nanonanavazade""]","[Black ""pepito333""]","[Result ""1-0""]","[UTCDate ""2020.02.01""]","[UTCTime ""00:05:44""]","[WhiteElo ""1633""]","[BlackElo ""1622""]","[WhiteRatingDiff ""+5""]"
4997,"[BlackRatingDiff ""-6""]","[ECO ""B02""]","[Opening ""Alekhine Defense: Scandinavian Varia...","[TimeControl ""120+1""]","[Termination ""Normal""]",1. e4 { [%clk 0:02:00] } d5 { [%clk 0:02:00] }...,"[Event ""Rated Blitz g